## **CREATE BLOB STORAGE**

[Setting up](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python)

[Documentation](https://docs.microsoft.com/en-us/python/api/azure-storage-blob/azure.storage.blob?view=azure-python)

In [1]:
import os, uuid, sys
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

### **1. Set up**

In [2]:
connect_str='XXX'
container_name ='containerstorage01'

### **2. Create container and folders**

In [3]:
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
try:
    container_client = blob_service_client.create_container(container_name)
except:
    container_client = blob_service_client.get_container_client(container_name)

### **3. Upload data**

#### 3.1 Upload files

In [4]:
def absoluteFilePaths(directory):
    """Return all files in the directory"""
    all_files = []
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            all_files.append(os.path.abspath(os.path.join(dirpath, f)))
    return all_files

def upload_all_files_to_container(connect_str, container_name, directory):
    """Upload all file from the local directory to Azure container."""
    all_files = absoluteFilePaths(directory)
    for file_path in all_files:
        try:
            file_path_azure = file_path.replace(os.path.abspath(directory), directory.replace('./', ''))
            blobclient = BlobClient.from_connection_string(connect_str, container_name, file_path_azure)
            with open(file_path, "rb") as data:
                blobclient.upload_blob(data)
        except:
            pass

In [5]:
upload_all_files_to_container(connect_str, container_name, './data')

#### 3.2 Upload directory

In [6]:
def upload_dir(connect_str, container_name, local_source, azure_destination):
    """
    Function which upload directory from the local directory to Container
    in Azure Storage if such not exists.
    
    sample inputs:
    connect_str = 'DefaultEndpointsProt...'
    container_name = syndicaistorage01
    local_source = './data'
    local_destination = 'data'
    """
    
    # Get container or create if not exists
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    try:
        container_client = blob_service_client.create_container(container_name)
    except:
        container_client = blob_service_client.get_container_client(container_name)
        
    # list all files in directory
    all_files = []
    for dirpath,_,filenames in os.walk(local_source):
        for f in filenames:
            all_files.append(os.path.abspath(os.path.join(dirpath, f)))
            
    # upload files
    for file_path in all_files:
        try:
            file_path_azure = '/'.join([azure_destination, file_path[len(os.path.abspath(local_source)) + 1:]])
            blobclient = BlobClient.from_connection_string(connect_str, container_name, file_path_azure)
            with open(file_path, "rb") as data:
                blobclient.upload_blob(data)
        except:
            pass

In [7]:
upload_dir(connect_str, container_name, './data', 'data')

#### **4. Download directory**

In [8]:
def download_dir(connect_str, container_name, directory, local_destination):
    """
    Function which download directory from the azure directory in Container 
    in Azure Storage to local destination.
    
    sample inputs:
    connect_str = 'DefaultEndpointsProt...'
    container_name = containerstorage01
    directory = 'data'
    local_destination = './data'
    """
    
    # Get container or create if not exists
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    try:
        container_client = blob_service_client.create_container(container_name)
    except:
        container_client = blob_service_client.get_container_client(container_name)
    
    # iterate over all files
    for blob in container_client.list_blobs(name_starts_with=directory):
        name = blob.name[len(directory) + 1:]
        # Check if blob.name is a directory create it locally
        if "/" in name:
            os.makedirs(os.path.join(local_destination, os.path.dirname(name)), exist_ok=True)
        
        # download file
        blobclient = BlobClient.from_connection_string(connect_str, container_name, blob.name)
        with open(os.path.join(local_destination, name), "wb") as download_file:
            download_file.write(blobclient.download_blob().readall())

In [9]:
download_dir(connect_str, container_name, 'data', './data2')